# Importations

In [3]:
!pip install pmdarima
!pip install XlsxWriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 4.4 MB/s eta 0:00:00


In [4]:
import numpy as np
from scipy.optimize import Bounds, minimize, SR1, LinearConstraint
from pandas import read_csv, DataFrame, concat
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor, ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
import pandas as pd
import pmdarima as pm
from copy import deepcopy
import scipy.optimize
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xlsxwriter as xls
from torch.autograd import Variable

#from LSTMseries import LSTMmodel

# Models

In [5]:
# transform a time series dataset into a supervised learning dataset
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
        df = DataFrame(data)
        cols = list()
        # input sequence (t-n, ... t-1)
        for i in range(n_in, 0, -1):
                cols.append(df.shift(i))
		# forecast sequence (t, t+1, ... t+n)
        for i in range(0, n_out):
                cols.append(df.shift(-i))
        # put it all together
        agg = concat(cols, axis=1)
        # drop rows with NaN values
        if dropnan:
                agg.dropna(inplace=True)
        return agg.values

In [6]:
"""# Prepare Training Data

For this example, we will use sliding windows of 10 points per each window (equ>

Min-Max scaling has also been fitted to the training data to aid the convergenc
"""

#def inverse_transform(y):
#  return target_scaler.inverse_transform(y.reshape(-1, 1))

def create_sliding_window(data, sequence_length, stride=1):
    X_list, y_list = [], []
    for i in range(len(data)):
      if (i + sequence_length) < len(data):
        X_list.append(data.iloc[i:i+sequence_length:stride, :].values)
        y_list.append(data.iloc[i+sequence_length, -1])
    return np.array(X_list), np.array(y_list)

In [7]:
class BayesianLSTM(nn.Module):

    def __init__(self, n_features, output_length, batch_size):

        super(BayesianLSTM, self).__init__()

        self.batch_size = batch_size # user-defined

        self.hidden_size_1 = 256 # number of encoder cells (from paper)
        self.hidden_size_2 = 64 # number of decoder cells (from paper)
        self.stacked_layers = 1 # number of (stacked) LSTM layers for each stage
        self.dropout_probability = 0 # arbitrary value (the paper suggests that>

        '''self.lstm1 = SFLSTM(input_size = n_features,
                            hidden_size = self.hidden_size_1,
                            num_layers=self.stacked_layers,
                            cell_size = self.hidden_size_1)'''
        self.lstm1 = nn.LSTM(n_features,
                             self.hidden_size_1,
                             num_layers=self.stacked_layers,
                             batch_first=True)
        self.lstm2 = nn.LSTM(self.hidden_size_1, self.hidden_size_2, num_layers=self.stacked_layers,
                             batch_first=True)
        self.fc = nn.Linear(self.hidden_size_2, output_length)
        self.loss_fn = nn.MSELoss()

    def forward(self, x):
        batch_size, seq_len, _ = x.size()

        hidden = self.init_hidden1(batch_size)
        output, _ = self.lstm1(x, hidden)
        output = F.dropout(output, p=self.dropout_probability, training=True)

        state = self.init_hidden2(batch_size)
        output, state = self.lstm2(output, state)
        output = F.dropout(output, p=self.dropout_probability, training=True)

        output = output[:, -1, :] # take the last decoder cell's outputs
        y_pred = self.fc(output)
        return y_pred

    def init_hidden1(self, batch_size):
        hidden_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_1))
        cell_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_1))
        return hidden_state, cell_state

    def init_hidden2(self, batch_size):
        hidden_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_2))
        cell_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_2))
        return hidden_state, cell_state

    def loss(self, pred, truth):
        return self.loss_fn(pred, truth)

    def predict(self, X):
        return self(torch.tensor(X, dtype=torch.float32)).view(-1).detach().numpy()

In [8]:
class BayesianDoubleLSTM(nn.Module):

    def __init__(self, n_features, output_length, batch_size):

        super(BayesianDoubleLSTM, self).__init__()

        self.batch_size = batch_size # user-defined

        self.hidden_size_1 = 64 # number of encoder cells (from paper)
        self.hidden_size_2 = 64 # number of decoder cells (from paper)
        self.stacked_layers = 1 # number of (stacked) LSTM layers for each stage
        self.dropout_probability = 0 # arbitrary value (the paper suggests that>

        self.lstm1 = nn.LSTM(n_features, self.hidden_size_1, num_layers=self.stacked_layers, batch_first=True)

        # self.lstm2 = nn.LSTM(self.hidden_size_1, self.hidden_size_2, num_layers=self.stacked_layers, batch_first=True)

        self.fc = nn.Linear(self.hidden_size_1, output_length)

        self.loss_fn = nn.MSELoss()

    def forward(self, x):
        batch_size, seq_len, _ = x.size()

        hidden = self.init_hidden1(batch_size)
        output, _ = self.lstm1(x, hidden)
        output = F.dropout(output, p=self.dropout_probability, training=True)

        # state = self.init_hidden2(batch_size)
        # output, state = self.lstm2(output, state)
        # output = F.dropout(output, p=self.dropout_probability, training=True)

        output = output[:, -1, :] # take the last decoder cell's outputs
        y_pred = self.fc(output)
        return y_pred

    def init_hidden1(self, batch_size):
        hidden_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_1))
        cell_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_1))
        return hidden_state, cell_state

    def init_hidden2(self, batch_size):
        hidden_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_2))
        cell_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_2))
        return hidden_state, cell_state

    def loss(self, pred, truth):
        return self.loss_fn(pred, truth)

    def predict(self, X):
        return self(torch.tensor(X, dtype=torch.float32)).view(-1).detach().numpy()

In [9]:
def LSTMmodel(filename):

    resample_df = pd.read_csv(filename)
    #resample_df = resample_df['tcm1yd']
    #resample_df = energy_df['Appliances']

    train_split = 0.7
    n_train = int(train_split * len(resample_df))
    n_test = len(resample_df) - n_train


    feature_array = resample_df.values

    if str(type(feature_array[0]))!="<class 'numpy.ndarray'>":
        feature_array = np.expand_dims(feature_array, axis = -1)

	  # Fit Scaler only on Training features
    feature_scaler = MinMaxScaler()
    feature_scaler.fit(feature_array[:n_train])
    # Fit Scaler only on Training target values
    target_scaler = MinMaxScaler()
    target_scaler.fit(feature_array[:n_train, -1].reshape(-1, 1))

    # Transfom on both Training and Test data
    scaled_array = pd.DataFrame(feature_scaler.transform(feature_array))

    sequence_length = 10
    X, y = create_sliding_window(scaled_array, sequence_length)

    X_train = X[:n_train]
    y_train = y[:n_train]
    X_test = X[n_train:]
    y_test = y[n_train:]


    #el mejor hasta ahora 256-32 (x2) dropout=0, lr=0.001, 1000 eps (0.039)

    n_features = scaled_array.shape[-1]
    sequence_length = 10
    output_length = 1

    batch_size = 128
    n_epochs = 25
    learning_rate = 0.01
    #weight_decay = learning_rate/n_epochs
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    bayesian_lstm = BayesianLSTM(n_features=n_features,
                                 output_length=output_length,
                                 batch_size = batch_size)
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(bayesian_lstm.parameters(), lr=learning_rate)
                               #  weight_decay=weight_decay)

    bayesian_lstm.train()

    print('fsfs')
    for e in range(1, n_epochs+1):
        for b in range(0, len(X_train), batch_size):
            features = X_train[b:b+batch_size]
            target = y_train[b:b+batch_size]

            X_batch = torch.tensor(features,dtype=torch.float32)
            y_batch = torch.tensor(target,dtype=torch.float32)

            output = bayesian_lstm(X_batch)
            loss = criterion(output.view(-1), y_batch)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        if e % 2 == 0:
          print('epoch', e, 'loss: ', loss.item())


    offset = sequence_length

    training_predictions = bayesian_lstm.predict(X_train)
    #training_df = inverse_transform(training_predictions)
    training_df = target_scaler.inverse_transform(training_predictions.reshape(-1,1))
    training_truth_df = resample_df.iloc[offset:n_train + offset:1]

    testing_predictions = bayesian_lstm.predict(X_test)
    #testing_df = inverse_transform(testing_predictions)
    testing_df = target_scaler.inverse_transform(testing_predictions.reshape(-1,1))
    testing_truth_df = resample_df.iloc[n_train + offset::1]

    y = np.array(testing_df)
    yp = np.array(testing_truth_df)
    ytr = np.array(training_df)
    ytrp = np.array(training_truth_df)
    err = mean_squared_error(y,yp)
    print('mse err: ', err)
    #err_train = mean_squared_error(ytr, ytrp)

    return ytr,y

In [10]:
def sliding_windows(data, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)

In [11]:
class simLSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(simLSTM, self).__init__()

        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)

        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))

        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))

        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))

        h_out = h_out.view(-1, self.hidden_size)

        out = self.fc(h_out)

        return out

In [13]:
def singleLSTMmodel(DATASET, learning_rate=0.01, num_epochs=1000):

  training_set = pd.read_csv(DATASET)

  training_set = training_set.iloc[:, 0].values
  training_set = np.expand_dims(training_set, 1)
  sc = MinMaxScaler()
  training_data = sc.fit_transform(training_set)

  seq_length = 6
  x, y = sliding_windows(training_data, seq_length)

  train_size = int(len(y) * 0.7)
  test_size = len(y) - train_size

  dataX = Variable(torch.Tensor(np.array(x)))
  dataY = Variable(torch.Tensor(np.array(y)))

  trainX = Variable(torch.Tensor(np.array(x[0:train_size])))
  trainY = Variable(torch.Tensor(np.array(y[0:train_size])))

  testX = Variable(torch.Tensor(np.array(x[train_size:len(x)])))
  testY = Variable(torch.Tensor(np.array(y[train_size:len(y)])))

  input_size = 1
  hidden_size = 64
  num_layers = 1

  num_classes = 1

  lstm = simLSTM(num_classes, input_size, hidden_size, num_layers)

  criterion = torch.nn.MSELoss()    # mean-squared error for regression
  optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
  #optimizer = torch.optim.SGD(lstm.parameters(), lr=learning_rate)

  # Train the model
  for epoch in range(num_epochs):
      outputs = lstm(trainX)
      optimizer.zero_grad()

      # obtain the loss function
      loss = criterion(outputs, trainY)

      loss.backward()

      optimizer.step()
      if epoch % 25 == 0:
        print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

  from sklearn.metrics import mean_squared_error
  lstm.eval()
  train_predict = lstm(dataX)

  data_predict = train_predict.data.numpy()
  dataY_plot = dataY.data.numpy()

  data_predict = sc.inverse_transform(data_predict)
  dataY_plot = sc.inverse_transform(dataY_plot)

  #plt.axvline(x=train_size, c='r', linestyle='--')

  #plt.plot(dataY_plot)
  #plt.plot(data_predict)
  #plt.suptitle('Time-Series Prediction')
  #plt.show()


  print('mse : ' , mean_squared_error(lstm(testX).data.numpy(), testY.data.numpy()))

  tf_trainX = sc.inverse_transform(lstm(trainX).data.numpy())
  tf_testX = sc.inverse_transform(lstm(testX).data.numpy())

  return tf_trainX, tf_testX

In [14]:
class simGRU(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(simGRU, self).__init__()

        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size

        self.lstm = nn.GRU(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True, dropout = 0.4)

        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))

        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))

        # Propagate input through LSTM
        ula, h_out = self.lstm(x, h_0)

        h_out = h_out.view(-1, self.hidden_size)

        out = self.fc(h_out)

        return out

In [15]:
def singleGRUmodel(DATASET, learning_rate=0.01, num_epochs=1000):

  training_set = pd.read_csv(DATASET)

  training_set = training_set.iloc[:, 0].values
  training_set = np.expand_dims(training_set, 1)
  sc = MinMaxScaler()
  training_data = sc.fit_transform(training_set)

  seq_length = 6
  x, y = sliding_windows(training_data, seq_length)

  train_size = int(len(y) * 0.7)
  test_size = len(y) - train_size

  dataX = Variable(torch.Tensor(np.array(x)))
  dataY = Variable(torch.Tensor(np.array(y)))

  trainX = Variable(torch.Tensor(np.array(x[0:train_size])))
  trainY = Variable(torch.Tensor(np.array(y[0:train_size])))

  testX = Variable(torch.Tensor(np.array(x[train_size:len(x)])))
  testY = Variable(torch.Tensor(np.array(y[train_size:len(y)])))

  input_size = 1
  hidden_size = 128
  num_layers = 1

  num_classes = 1

  lstm = simGRU(num_classes, input_size, hidden_size, num_layers)

  criterion = torch.nn.MSELoss()    # mean-squared error for regression
  optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
  #optimizer = torch.optim.SGD(lstm.parameters(), lr=learning_rate)

  # Train the model
  for epoch in range(num_epochs):
      outputs = lstm(trainX)
      optimizer.zero_grad()

      # obtain the loss function
      loss = criterion(outputs, trainY)

      loss.backward()

      optimizer.step()
      if epoch % 100 == 0:
        print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

  from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
  lstm.eval()
  train_predict = lstm(dataX)

  data_predict = train_predict.data.numpy()
  dataY_plot = dataY.data.numpy()

  data_predict = sc.inverse_transform(data_predict)
  dataY_plot = sc.inverse_transform(dataY_plot)

  plt.axvline(x=train_size, c='r', linestyle='--')

  plt.plot(dataY_plot)
  plt.plot(data_predict)
  plt.suptitle('Time-Series Prediction')
  plt.show()


  print('mse : ' , mean_squared_error(testY.data.numpy(), lstm(testX).data.numpy()))
  print('mae : ' , mean_absolute_error(testY.data.numpy(), lstm(testX).data.numpy()))
  print('r2 : ' , r2_score(testY.data.numpy(), lstm(testX).data.numpy()))

  tf_trainX = sc.inverse_transform(lstm(trainX).data.numpy())
  tf_testX = sc.inverse_transform(lstm(testX).data.numpy())

  return tf_trainX, tf_testX

In [16]:
def LSTMmodel2(filename):

    resample_df = pd.read_csv(filename)
    #resample_df = resample_df['tcm1yd']
    #resample_df = energy_df['Appliances']

    train_split = 0.7
    n_train = int(train_split * len(resample_df))
    n_test = len(resample_df) - n_train


    feature_array = resample_df.values

    if str(type(feature_array[0]))!="<class 'numpy.ndarray'>":
        feature_array = np.expand_dims(feature_array, axis = -1)

	  # Fit Scaler only on Training features
    feature_scaler = MinMaxScaler()
    feature_scaler.fit(feature_array[:n_train])
    # Fit Scaler only on Training target values
    target_scaler = MinMaxScaler()
    target_scaler.fit(feature_array[:n_train, -1].reshape(-1, 1))

    # Transfom on both Training and Test data
    scaled_array = pd.DataFrame(feature_scaler.transform(feature_array))

    sequence_length = 10
    X, y = create_sliding_window(scaled_array, sequence_length)

    X_train = X[:n_train]
    y_train = y[:n_train]
    X_test = X[n_train:]
    y_test = y[n_train:]


    #el mejor hasta ahora 256-32 (x2) dropout=0, lr=0.001, 1000 eps (0.039)

    n_features = scaled_array.shape[-1]
    sequence_length = 10
    output_length = 1

    batch_size = 128
    n_epochs = 700
    learning_rate = 0.01
    #weight_decay = learning_rate/n_epochs
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    bayesian_lstm = BayesianDoubleLSTM(n_features=n_features,
                                 output_length=output_length,
                                 batch_size = batch_size)
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(bayesian_lstm.parameters(), lr=learning_rate)
                               #  weight_decay=weight_decay)

    bayesian_lstm.train()

    for e in range(1, n_epochs+1):
        for b in range(0, len(X_train), batch_size):
            features = X_train[b:b+batch_size]
            target = y_train[b:b+batch_size]

            X_batch = torch.tensor(features,dtype=torch.float32)
            y_batch = torch.tensor(target,dtype=torch.float32)

            output = bayesian_lstm(X_batch)
            loss = criterion(output.view(-1), y_batch)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        if e % 50 == 0:
          print('epoch', e, 'loss: ', loss.item())


    offset = sequence_length

    training_predictions = bayesian_lstm.predict(X_train)
    #training_df = inverse_transform(training_predictions)
    training_df = target_scaler.inverse_transform(training_predictions.reshape(-1,1))
    training_truth_df = resample_df.iloc[offset:n_train + offset:1]

    testing_predictions = bayesian_lstm.predict(X_test)
    #testing_df = inverse_transform(testing_predictions)
    testing_df = target_scaler.inverse_transform(testing_predictions.reshape(-1,1))
    testing_truth_df = resample_df.iloc[n_train + offset::1]

    y = np.array(testing_df)
    yp = np.array(testing_truth_df)
    ytr = np.array(training_df)
    ytrp = np.array(training_truth_df)
    #err = mean_squared_error(y,yp)
    #err_train = mean_squared_error(ytr, ytrp)

    return ytr,y

In [17]:
def RFmodel(filename):
	#filename = 'dataset_2.csv'
    datos = read_csv(filename)
    datos = datos[datos.columns[0]]
    #datos=archivo["tcm1yd"]
    datos = [float(numeric_string) for numeric_string in datos]

    train_split = 0.7
    n_train = int(train_split * len(datos))
    n_test = len(datos) - n_train

    data = series_to_supervised(datos, n_in=6)
    trainX, trainY = data[:n_train, :-1], data[:n_train, -1]
    testX, testY = data[n_train:, :-1], data[n_train:, -1]
    model = RandomForestRegressor(n_estimators=1000)
    model.fit(trainX, trainY)

    yhat = model.predict(testX)
    yhattrain = model.predict(trainX)
    return yhattrain, yhat

In [18]:
def GBmodel (filename):
    #filename = 'dataset_2.csv'
    datos = read_csv(filename)
    datos = datos[datos.columns[0]]
    #datos=archivo["Appliances"]
    datos = [float(numeric_string) for numeric_string in datos]

    train_split = 0.7
    n_train = int(train_split * len(datos))
    n_test = len(datos) - n_train
	 # transform a time series dataset into a supervised learning dataset
    n_in = 6
    data = series_to_supervised(datos, n_in=n_in)
    trainX_GB, trainY_GB = data[:n_train, :-1], data[:n_train, -1]
    testX_GB, testY_GB = data[n_train:, :-1], data[n_train:, -1]
    model = GradientBoostingRegressor(n_estimators=1000)
    model.fit(trainX_GB, trainY_GB)

    yhat_GB = model.predict(testX_GB)
    yhattrain_GB = model.predict(trainX_GB)
    return yhattrain_GB, yhat_GB

In [19]:
def BRmodel (filename):
    #filename = 'dataset_2.csv'
    datos = read_csv(filename)
    datos = datos[datos.columns[0]]
    #datos=archivo["Appliances"]
    datos = [float(numeric_string) for numeric_string in datos]

    train_split = 0.7
    n_train = int(train_split * len(datos))
    n_test = len(datos) - n_train
	 # transform a time series dataset into a supervised learning dataset
    n_in = 6
    data = series_to_supervised(datos, n_in=n_in)
    trainX_GB, trainY_GB = data[:n_train, :-1], data[:n_train, -1]
    testX_GB, testY_GB = data[n_train:, :-1], data[n_train:, -1]
    model = BaggingRegressor(n_estimators=1000)
    model.fit(trainX_GB, trainY_GB)

    yhat_BR = model.predict(testX_GB)
    yhattrain_BR = model.predict(trainX_GB)
    return yhattrain_BR, yhat_BR

In [20]:
def KNNmodel (filename):
    #filename = 'dataset_2.csv'
    datos = read_csv(filename)
    datos = datos[datos.columns[0]]
    #datos=archivo["Appliances"]
    datos = [float(numeric_string) for numeric_string in datos]

    train_split = 0.7
    n_train = int(train_split * len(datos))
    n_test = len(datos) - n_train
	 # transform a time series dataset into a supervised learning dataset
    n_in = 6
    data = series_to_supervised(datos, n_in=n_in)
    trainX_GB, trainY_GB = data[:n_train, :-1], data[:n_train, -1]
    testX_GB, testY_GB = data[n_train:, :-1], data[n_train:, -1]
    model = KNeighborsRegressor(n_neighbors=3)
    model.fit(trainX_GB, trainY_GB)

    yhat_KNN = model.predict(testX_GB)
    yhattrain_KNN = model.predict(trainX_GB)
    return yhattrain_KNN, yhat_KNN

In [21]:
def ARIMAmodel(filename):
    datos = pd.read_csv(filename)
    #datos=archivo["Appliances"]
    datos = datos[datos.columns[0]]
    data = [float(numeric_string) for numeric_string in datos]

    train_split = 0.7
    n_train = int(train_split * len(datos))
    n_test = len(datos) - n_train

    train, test =data[:n_train], data[n_train:]
    test = test[:n_test]
    # Fit a simple auto_arima model
    arima = pm.auto_arima(train, error_action='ignore', trace=True,
                          suppress_warnings=True, maxiter=5,
                          seasonal=False, m=12)

    pred_arima_train=arima.predict_in_sample()
    new_model = deepcopy(arima)
    new_model.method = "nm"
    new_model.fit(test, maxiter=0, start_params=new_model.params())
    pred_arima_test=new_model.predict_in_sample()
    pred_arima_test = pred_arima_test[6:]
    return pred_arima_train, pred_arima_test

# Ordering & Weight calculation

In [22]:
def ordenar(X,y): #X es el array con todas las predicciones, modelos en filas, >
    n = len(X[0])
    Z = X.copy()
    for i in range(1,n):
        O = np.maximum(1-np.abs(X[:,i-1]-y[i-1])/np.abs(y[i-1]),0)
        SO = np.argsort(O)
        for k in SO:
            Z[k,i] = np.mean(X[SO[k],i])#Esto es porque se coge la media si hay>
    return(Z[:,1:]) #el primer tiempo me lo cargo (no lo puedo ordenar)

In [23]:
def ordenarPorValor(X): #X es el array con todas las predicciones, modelos en filas, >
    n = len(X[0])
    Z = X.copy()
    for i in range(n):
        O = np.maximum(X[:,i-1],0)
        SO = np.argsort(O)
        for k in SO:
            Z[k,i] = np.mean(X[SO[k],i])#Esto es porque se coge la media si hay>
    return(Z) #el primer tiempo me lo cargo (no lo puedo ordenar)

In [24]:
def ECM(w,Z,y):
    ECM = 0
    y = np.transpose(y)
    for i in range(len(y)):
        ECM += (np.dot(w,Z[:,i])-y[i])**2
    return(ECM/len(y))

In [25]:
def pesosIOWA(X,y):
    Z = ordenar(X,y)
    m = len(Z)
    bounds = Bounds([0]*m,[1]*m)
    linear_constraint = LinearConstraint([1]*m, [1], [1])
    x0 = np.array([1./m]*m)
    res_IOWA = minimize(ECM, x0, method='trust-constr',  jac="2-point", hess=SR1(), args=(Z,np.transpose(y[1:])), options={'maxiter': 1000}, bounds=bounds, constraints=linear_constraint)
    return(res_IOWA.x)

In [26]:
def pesosOWA(X,y):
    Z = ordenarPorValor(X)
    m = len(Z)
    bounds = Bounds([0]*m,[1]*m)
    linear_constraint = LinearConstraint([1]*m, [1], [1])
    x0 = np.array([1./m]*m)
    res_IOWA = minimize(ECM, x0, method='trust-constr',  jac="2-point", hess=SR1(), args=(Z,np.transpose(y)), options={'maxiter': 1000}, bounds=bounds, constraints=linear_constraint)
    return(res_IOWA.x)

In [ ]:
def pesosOWA1(X,y):
    Z = ordenarPorValor(X)
    m = len(Z)
    bounds = Bounds([0]*m,[1]*m)
    linear_constraint = LinearConstraint([1]*m, [1], [1])
    x0 = np.array([1./m]*m)
    res_IOWA = minimize(ECM, x0, method='trust-constr',  jac="2-point", hess=SR1(), args=(Z,np.transpose(y[1:])), options={'maxiter': 1000}, bounds=bounds, constraints=linear_constraint)
    return(res_IOWA.x)

In [28]:
def pesosNOWA(X,y):
    Z = ordenarPorValor(X)
    m = len(Z)
    linear_constraint = LinearConstraint([1]*m, [1], [1])
    x0 = np.array([1./m]*m)
    res_IOWA = minimize(ECM, x0, method='trust-constr',  jac="2-point", hess=SR1(), args=(Z,np.transpose(y)), options={'maxiter': 1000},constraints=linear_constraint)
    return(res_IOWA.x)

In [29]:
def pesosNWAM(X,y):
    #Z = ordenar(X,y)
    m = len(X)
    linear_constraint = LinearConstraint([1]*m, [1], [1])
    x0 = np.array([1./m]*m)
    res_IOWA = minimize(ECM, x0, method='trust-constr',  jac="2-point", hess=SR1(), args=(X,np.transpose(y)), options={'maxiter': 1000}, constraints=linear_constraint)
    return(res_IOWA.x)

In [30]:
def pesosWAM(X,y):
    #Z = ordenar(X,y)
    m = len(X)
    bounds = Bounds([0]*m,[1]*m)
    linear_constraint = LinearConstraint([1]*m, [1], [1])
    x0 = np.array([1./m]*m)
    res_IOWA = minimize(ECM, x0, method='trust-constr',  jac="2-point", hess=SR1(), args=(X,np.transpose(y)), options={'maxiter': 1000}, bounds=bounds, constraints=linear_constraint)
    return(res_IOWA.x)

In [31]:
def pesosIOLF(X,y):
    Z = ordenar(X,y)
    m = len(Z)
    linear_constraint = LinearConstraint([1]*m, [1], [1])
    x0 = np.array([1./m]*m)
    res_IOLF = minimize(ECM, x0, method='trust-constr',  jac="2-point", hess=SR1(), args=(Z,np.transpose(y[1:])), options={'verbose': 1}, constraints=linear_constraint)
    return(res_IOLF.x)

# Auxiliar functions

In [32]:
def read_data(Folder,ith):
    dataset_name = str(Folder) + "/" + str(ith) + ".csv"
    data = read_csv(dataset_name)
    data = data[data.columns[0]]
    data = [float(numeric_string) for numeric_string in data]
    return data, dataset_name

In [33]:
def split_data(data, train_split=0.7):
    n_train = int(train_split * len(data))
    n_test = len(data) - n_train

    data = series_to_supervised(data, n_in=6)
    trainX, ytrain = data[:n_train, :-1], data[:n_train, -1]
    testX, ytest = data[n_train:, :-1], data[n_train:, -1]

    return ytrain, ytest


In [34]:
def predictions(dataset_name):
    preds = [RFmodel(dataset_name), GBmodel(dataset_name), ARIMAmodel(dataset_name), KNNmodel(dataset_name),
            BRmodel(dataset_name), singleLSTMmodel(dataset_name, num_epochs=1200), singleGRUmodel(dataset_name, num_epochs=1200)]

    aux1 = preds[5][0].squeeze(axis = 1)
    aux2 = preds[5][1].squeeze(axis = 1)
    aux3 = preds[6][0].squeeze(axis = 1)
    aux4 = preds[6][1].squeeze(axis = 1)

    preds[5] = [aux1, aux2]
    preds[6] = [aux3, aux4]

    return preds


In [35]:
def stack_predictions(preds):
    Xtrain = np.vstack((preds[0][0][:-5] , preds[1][0][:-5], preds[2][0][:-5], preds[3][0][:-5], preds[4][0][:-5], preds[5][0], preds[6][0]))
    # Xtest = np.vstack((preds[0][1][4:], preds[1][1][4:], preds[2][1][4:], preds[3][1][4:], preds[4][1][4:], preds[5][1], preds[6][1]))
    Xtest = np.vstack((preds[0][1], preds[1][1], preds[2][1], preds[3][1], preds[4][1], preds[5][1][4:], preds[6][1][4:]))
    # Xtest = np.vstack((preds[0][1][4:], preds[1][1], preds[2][1][4:], preds[3][1][4:], preds[4][1][4:], preds[5][1], preds[6][1]))
    return Xtrain, Xtest


In [45]:
def Xy_transform(Xtrain, ytrain, Xtest, ytest):
    Xtrain_transformed = []
    ytrain_transformed = ytrain[:-12]
    for i in range(0,7):
        if i == 2:
            Xtrain_transformed.append(Xtrain[i][12-5:])
        else:
            Xtrain_transformed.append(Xtrain[i][1:-6])
    Xtest_transformed = []
    ytest_transformed = ytest[:-2]
    for i in range(0,7):
        if i == 5 or i==6:
            Xtest_transformed.append(Xtest[i][2:])
        else:
            Xtest_transformed.append(Xtest[i][1:-1])
    return np.array(Xtrain_transformed), ytrain_transformed, np.array(Xtest_transformed), ytest_transformed

#Errors


In [36]:
def individual_errors(preds, ytest):
    MSE = []
    MAE = []
    R2 = []
    #a = preds[1][1].squeeze(axis = 1)
    #preds[1] = [preds[1][0], a]
    for i in range(5):
        if i == 9:
            MSE.append(mean_squared_error(ytest[5:], preds[i][1][1:]))
            MAE.append(mean_absolute_error(ytest[5:], preds[i][1][1:]))
            R2.append(r2_score(ytest[5:], preds[i][1][1:]))
        else:
            MSE.append(mean_squared_error(ytest, preds[i][1]))
            MAE.append(mean_absolute_error(ytest, preds[i][1]))
            R2.append(r2_score(ytest, preds[i][1]))
    for j in range(5,7):
        MSE.append(mean_squared_error(ytest, preds[j][1][4:]))
        MAE.append(mean_absolute_error(ytest, preds[j][1][4:]))
        R2.append(r2_score(ytest, preds[j][1][4:]))

    errors = {'MSE': MSE, 'MAE': MAE, 'R2': R2}
    return errors

In [37]:
def individual_errors_byX(Xtest_transformed, ytest_transformed):
    MSE = []
    MAE = []
    R2 = []
    for i in range(0,7):
        MSE.append(mean_squared_error(ytest_transformed, Xtest_transformed[i]))
        MAE.append(mean_absolute_error(ytest_transformed, Xtest_transformed[i]))
        R2.append(r2_score(ytest_transformed, Xtest_transformed[i] ))
    errors = {'MSE': MSE, 'MAE': MAE, 'R2': R2}
    return errors

In [38]:
def time_ensemble_error(weights, Ztest, ytest, operator, err_type='MSE'):

    if err_type == 'MSE':
        err = mean_squared_error(np.dot(weights[operator], Ztest), ytest  )

    if err_type == 'MAE':
        err = mean_absolute_error(np.dot(weights[operator], Ztest), ytest)

    if err_type == 'R2':
        err = r2_score(np.dot(weights[operator], Ztest), ytest)

    return err

In [39]:
def ensemble_errors(weights, Ztest, Xtest, ytest):
    Zo = Ztest
    yo = ytest[1:]
    Z = Xtest
    y = ytest
    C = ordenarPorValor(Xtest)
    ensemble_errors = {
        'MSE': {
            'IOLF': time_ensemble_error(weights, Zo, yo, 'IOLF'),
            'IOWA': time_ensemble_error(weights, Zo, yo, 'IOWA'),
            'WAM':  time_ensemble_error(weights, Z, y, 'WAM'),
            'OWA': time_ensemble_error(weights, C, y, 'OWA'),
            'NWAM':  time_ensemble_error(weights, Z, y, 'NWAM'),
            'NOWA': time_ensemble_error(weights, C, y, 'NOWA')
        },
        'MAE': {
            'IOLF': time_ensemble_error(weights, Zo, yo, 'IOLF', 'MAE'),
            'IOWA': time_ensemble_error(weights, Zo, yo, 'IOWA', 'MAE'),
            'WAM':  time_ensemble_error(weights, Z, y, 'WAM', 'MAE'),
            'OWA': time_ensemble_error(weights, C, y, 'OWA', 'MAE'),
            'NWAM':  time_ensemble_error(weights, Z, y, 'NWAM', 'MAE'),
            'NOWA': time_ensemble_error(weights, C, y, 'NOWA', 'MAE')
        },
        'R2': {
            'IOLF': time_ensemble_error(weights, Zo, yo, 'IOLF', 'R2'),
            'IOWA': time_ensemble_error(weights, Zo, yo, 'IOWA', 'R2'),
            'WAM':  time_ensemble_error(weights, Z, y, 'WAM', 'R2'),
            'OWA': time_ensemble_error(weights, C, y, 'OWA', 'R2'),
            'NWAM':  time_ensemble_error(weights, Z, y, 'NWAM', 'R2'),
            'NOWA': time_ensemble_error(weights, C, y, 'NOWA', 'R2')
        }
    }
    return ensemble_errors

In [43]:
import itertools
def err_val (errors):
    vals = list(errors.values())
    vals[0][7] = list(vals[0][7].values())
    vals[1][7] = list(vals[1][7].values())
    vals[2][7] = list(vals[2][7].values())
    return list(itertools.chain(vals[0][:7],vals[0][7], vals[1][:7],vals[1][7], vals[2][:7],vals[2][7] ))

#Main

In [ ]:
weights_IOLF = []
weights_IOWA = []
operators =['IOLF', 'IOWA', 'WAM', 'OWA', 'NWAM', 'NOWA']
MSE_IOLF = []
MSE_IOWA = []
MAE_IOLF = []
MAE_IOWA = []
errors = []
ds_pos = []

ds_ini = 33
ds_fin = 50

excel = xls.Workbook('Aligned-results-' + str(ds_ini) + '-' + str(ds_fin) + '.xlsx')
sheet_test = excel.add_worksheet()
sheet_test.set_column('A:A', 20)
sheet_train = excel.add_worksheet()
sheet_train.set_column('A:A', 20)

errors_matrix = []
train_errors_matrix = []
err_types = ['MSE', 'MAE', 'R2']
header = []

for j in range(len(err_types)):
    for i in range(7+len(operators)):
        if i<7:
            s = err_types[j] + ' ' + str(i)
            header.append(s)
        else:
            s = err_types[j] + ' ' + operators[i - 7]
            header.append(s)
header_vec = [{'header': header[i]} for i in range(len(header))]

# range_datasets = [i for i in chain(range(1,5), range(25,29), range(33,50))]

for dataset in range(ds_ini, ds_fin +1):
    for reps in range(5):

        data, dataset_name = read_data('Datasets', dataset)

        ytrain, ytest = split_data(data)

        preds = predictions(dataset_name)

        Xtrain, Xtest = stack_predictions(preds)

        Xtrain_transformed, ytrain_transformed, Xtest_transformed, ytest_transformed = Xy_transform(Xtrain, ytrain, Xtest, ytest)

        weights = {
                    'IOLF': pesosIOLF(Xtrain_transformed, ytrain_transformed),
                    'IOWA': pesosIOWA(Xtrain_transformed, ytrain_transformed),
                    'WAM':  pesosWAM(Xtrain_transformed, ytrain_transformed),
                    'OWA':  pesosOWA(Xtrain_transformed, ytrain_transformed),
                    'NWAM': pesosNWAM(Xtrain_transformed, ytrain_transformed),
                    'NOWA': pesosNOWA(Xtrain_transformed, ytrain_transformed)
                    }


        Ztest, Ztrain = ordenar(Xtest_transformed, ytest_transformed), ordenar(Xtrain_transformed, ytrain_transformed)
        # TEST ERRORS
        errors = individual_errors_byX(Xtest_transformed, ytest_transformed)
        ens_errors = ensemble_errors(weights, Ztest, Xtest_transformed, ytest_transformed)
        errors['MSE'].append(ens_errors['MSE'])
        errors['MAE'].append(ens_errors['MAE'])
        errors['R2'].append(ens_errors['R2'])

        errors_matrix.append(err_val(errors))

        # TRAIN ERRORS
        train_errors = individual_errors_byX(Xtrain_transformed, ytrain_transformed)
        train_ens_errors = ensemble_errors(weights, Ztrain, Xtrain_transformed, ytrain_transformed)
        train_errors['MSE'].append(train_ens_errors['MSE'])
        train_errors['MAE'].append(train_ens_errors['MAE'])
        train_errors['R2'].append(train_ens_errors['R2'])

        train_errors_matrix.append(err_val(train_errors))


        weights_IOLF.append(weights['IOLF'])
        weights_IOWA.append(weights['IOWA'])


        ds_pos.append(dataset_name)


        #sheet.write(1, 0, err_MSE_IOLF)
        #sheet.write(1, 1, err_MSE_IOWA)
        #sheet.write(1, 2, err_MAE_IOLF)
        #sheet.write(1, 3, err_MAE_IOWA)
        print('Dataset ' + str(dataset))
        print('Test: ')
        print(errors_matrix)
        print('Train: ')
        print(train_errors_matrix)
        print(str(weights['IOLF']), str(weights['IOWA']))


sheet_test.add_table('A1:AM'+str(len(errors_matrix)+1),
                {'data': errors_matrix,
                 'columns': header_vec} )

sheet_train.add_table('A1:AM'+str(len(train_errors_matrix)+1),
                {'data': train_errors_matrix,
                 'columns': header_vec} )
excel.close()

In [ ]:
'''weights_IOLF = []
weights_IOWA = []
operators =['IOLF', 'IOWA', 'WAM', 'OWA']
MSE_IOLF = []
MSE_IOWA = []
MAE_IOLF = []
MAE_IOWA = []
errors = []
ds_pos = []

excel = xls.Workbook('results.xlsx')
sheet = excel.add_worksheet()
sheet.set_column('A:A', 20)

errors_matrix = []
err_types = ['MSE', 'MAE', 'R2']
header = []
for j in range(len(err_types)):
    for i in range(7+len(operators)):
        if i<7:
            s = err_types[j] + ' ' + str(i)
            header.append(s)
        else:
            s = err_types[j] + ' ' + operators[i - 7]
            header.append(s)
header_vec = [{'header': header[i]} for i in range(len(header))]

for dataset in range(33,38):
    for reps in range(1):

        data, dataset_name = read_data('Datasets', dataset)

        ytrain, ytest = split_data(data)

        preds = predictions(dataset_name)

        Xtrain, Xtest = stack_predictions(preds)

        weights = {
                    'IOLF': pesosIOLF(Xtrain, ytrain[:-5]),
                    'IOWA': pesosIOWA(Xtrain, ytrain[:-5]),
                    'WAM':  pesosWAM(Xtrain, ytrain[:-5]),
                    'OWA':  pesosOWA(Xtrain, ytrain[:-5])
                    }


        Ztest, Ztrain = ordenar(Xtest, ytest), ordenar(Xtrain, ytrain)

        errors = individual_errors(preds, ytest)
        ens_errors = ensemble_errors(weights, Ztest, ytest)
        errors['MSE'].append(ens_errors['MSE'])
        errors['MAE'].append(ens_errors['MAE'])
        errors['R2'].append(ens_errors['R2'])

        errors_matrix.append(err_val(errors))

        weights_IOLF.append(weights['IOLF'])
        weights_IOWA.append(weights['IOWA'])


        ds_pos.append(dataset_name)


        #sheet.write(1, 0, err_MSE_IOLF)
        #sheet.write(1, 1, err_MSE_IOWA)
        #sheet.write(1, 2, err_MAE_IOLF)
        #sheet.write(1, 3, err_MAE_IOWA)
        print(errors_matrix)
        print(str(weights['IOLF']), str(weights['IOWA']))


sheet.add_table('A1:AG'+str(len(errors_matrix)+1),
                {'data': errors_matrix,
                 'columns': header_vec} )
excel.close()'''